<a href="https://colab.research.google.com/github/mxbb-0-1/20MIS1068_AI/blob/main/Testing_the_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification
import wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
test_descriptions = [
    "Solar-powered lawn mowers",
    "Artificial intelligence software for medical imaging",
    "Bluetooth headphones with noise cancellation",
    "VR gaming consoles",
    "Augmented reality glasses",
    "Quantum computing hardware",
    "Self-driving car systems",
    "3D-printed jewelry",
    "Robotics kits for children",
    "Cloud-based storage solutions"
]


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Assuming you already have the tokenizer from the previous steps
test_inputs = tokenizer(test_descriptions, padding=True, truncation=True, return_tensors='pt')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import json
import pandas as pd

with open('idmanual.json', 'r') as file:
    data = json.load(file)

# Convert the JSON data to a pandas DataFrame
df = pd.DataFrame(data)
num_labels = len(df['class_id'].unique())


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

model.load_state_dict(torch.load("/content/drive/MyDrive/trademark_classification_model.pth"))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.eval()  # Set the model to evaluation mode


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
descriptions = df['description'].values
statuses = df['status'].map({'A': 1, 'D': 0}).values
class_ids = df['class_id'].values

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['class_id_encoded'] = label_encoder.fit_transform(df['class_id'])
df['status_encoded'] = df['status'].map({'A': 1, 'D': 0})

In [ ]:

import time

start_time = time.time()
with torch.no_grad():
    test_outputs = model(test_inputs['input_ids'].to(device), attention_mask=test_inputs['attention_mask'].to(device))
    logits = test_outputs.logits
    predictions = torch.argmax(logits, dim=1)

predicted_classes = label_encoder.inverse_transform(predictions.cpu().numpy())
for description, predicted_class in zip(test_descriptions, predicted_classes):
    print(f"Description: {description}\nPredicted Class: {predicted_class}\n")

# Perform inference (as shown above)
end_time = time.time()

inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.4f} seconds")


Description: Solar-powered lawn mowers
Predicted Class: 007

Description: Artificial intelligence software for medical imaging
Predicted Class: 009

Description: Bluetooth headphones with noise cancellation
Predicted Class: 009

Description: VR gaming consoles
Predicted Class: 028

Description: Augmented reality glasses
Predicted Class: 009

Description: Quantum computing hardware
Predicted Class: 009

Description: Self-driving car systems
Predicted Class: 012

Description: 3D-printed jewelry
Predicted Class: 014

Description: Robotics kits for children
Predicted Class: 009

Description: Cloud-based storage solutions
Predicted Class: 042

Inference Time: 0.8422 seconds


In [ ]:
wandb.init(project="trademark-class-prediction")
wandb.log({
    "test_predictions": predicted_classes.tolist(),
    "inference_time": inference_time
})


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
